In [ ]:
import os,sys
import kfp
from dkube.pipelines import dkube_training_op, dkube_serving_op
from dkube.sdk import DkubeApi, DkubeTraining, DkubeServing

In [ ]:
username = os.getenv("USERNAME")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
api = DkubeApi(token=token)
run_id = 0

In [ ]:
code_name = "sklearn-examples"
dataset = "titanic"
model_name = "titanic"

In [ ]:
training = DkubeTraining(username, name="train")
training.update_container(framework="scikit_0.2.2")
training.add_code(code_name, commitid="")
training.update_startupscript("python titanic/train.py")
training.add_input_dataset(dataset, mountpath='/titanic')
training.add_output_model(model_name, mountpath='/model')


serving = DkubeServing(username, name="serving")
serving.set_transformer(True, script='titanic/transformer.py')
serving.update_serving_model(model_name)
    
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='sample titanic pipeline with dkube components'
)
def titanic_pipeline(token, program='training'):
    train   = dkube_training_op("train", authtoken=token, training=training)    
    dkube_serving_op("serving", authtoken=token, serving=serving).after(train)

In [ ]:
client.create_run_from_pipeline_func(titanic_pipeline, run_name="[titanic] Run" + str(runid), arguments={"token":token})
runid += 1

In [ ]:
#kfp.compiler.Compiler().compile(titanic_pipeline, "titanic_pipeline.zip")